In [2]:
import torch
import torch.nn as nn
from torchvision import models

def pretrain_model(model_name='resnet', freeze_percent=0.5, freeze_all_except_last_layer='No'):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Loading the models.........................
    if model_name == 'resnet':
        model = models.resnet50(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
        classifier_layer_name = 'fc'

    elif model_name == 'googlenet':
        model = models.googlenet(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
        classifier_layer_name = 'fc'

    elif model_name == 'inception':
        model = models.inception_v3(pretrained=True, aux_logits=False)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
        classifier_layer_name = 'fc'

    elif model_name == 'vgg':
        model = models.vgg16(pretrained=True)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 10)
        classifier_layer_name = 'classifier.6'

    else:
        raise ValueError("Unsupported model. Choose from ['resnet', 'googlenet', 'inception', 'vgg']")

    # Freezing stratagies...........................
    if freeze_all_except_last_layer == 'Yes':
        for name, param in model.named_parameters():
            if classifier_layer_name not in name:
                param.requires_grad = False
    else:
        # Partial freezing: Freeze % of the model
        children = list(model.children())
        num_layers = len(children)
        num_freeze = int(freeze_percent * num_layers)

        for idx, layer in enumerate(children):
            requires_grad = False if idx < num_freeze else True
            for param in layer.parameters():
                param.requires_grad = requires_grad

    return model.to(device)


# 1. Freeze all except last layer
model = pretrain_model(model_name='resnet', freeze_percent=0.5, freeze_all_except_last_layer='Yes')
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 189MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, random_split
import numpy as np

def load_dataloaders(train_data_dir, data_aug='Yes', batch_size=64, val_split=0.2):
    # Define image transformations
    basic_resize = transforms.Resize((224, 224))
    to_tensor = transforms.ToTensor()
    normalize = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    crop_resize = transforms.RandomResizedCrop(224)
    flip = transforms.RandomHorizontalFlip()
    color_shift = transforms.ColorJitter(0.2, 0.2, 0.2, 0.1)
    rotate = transforms.RandomRotation(20)

    # Apply augmentation or not
    if data_aug == 'Yes':
        transform_pipeline = transforms.Compose([
            crop_resize,
            color_shift,
            flip,
            rotate,
            to_tensor,
            normalize
        ])
    else:
        transform_pipeline = transforms.Compose([
            basic_resize,
            to_tensor,
            normalize
        ])

    # Load dataset
    full_dataset = ImageFolder(root=train_data_dir, transform=transform_pipeline)
    total_len = len(full_dataset)
    val_len = int(val_split * total_len)
    train_len = total_len - val_len

    # Split the dataset using torch's random_split
    train_set, val_set = random_split(full_dataset, [train_len, val_len], generator=torch.Generator().manual_seed(42))

    # Create dataloaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)

    return {
        'train': train_loader,
        'val': val_loader,
        'classes': full_dataset.classes
    }


In [5]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def train_with_wandb():
    wandb.init(project="MA23M021_A2_Part-B", name="run-" + wandb.util.generate_id())
    config = wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Loading pretrained model...................................
    model = pretrain_model(
        model_name=config.model_name,                       
        freeze_percent=config.freeze_percent,             
        #freeze_all_except_last_layer=config.freeze_all_except_last_layer 
    )

    # Loading data...............................
    dataloaders = load_dataloaders(
        train_data_dir='/kaggle/input/natrue-12k/inaturalist_12K/train',
        data_aug=config.data_aug,
        batch_size=64,
        val_split=0.2
    )

    train_loader = dataloaders['train']
    val_loader = dataloaders['val']


    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.lr)

    for epoch in range(config.epochs):
        model.train()
        train_loss, train_correct, total = 0.0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            total += labels.size(0)
            train_correct += (preds == labels).sum().item()

        train_acc = 100 * train_correct / total
        train_loss /= total

        # Validation on val_data...........................
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                val_outputs = model(val_images)
                loss = criterion(val_outputs, val_labels)

                val_loss += loss.item() * val_images.size(0)
                _, val_preds = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += (val_preds == val_labels).sum().item()

        val_acc = 100 * val_correct / val_total
        val_loss /= val_total

        # Loging to wandb...................................
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })

        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    wandb.finish()


In [6]:
sweep_config = {
    'method': 'bayes',
    #'name': 'Pretrained-Finetune-sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
        'freeze_percent': {
            'values': [0.8, 0.9]
        },
        #'freeze_all_except_last_layer': {
        #    'values': ['Yes', 'No']
        #},
        'model_name': {
            'values': ['resnet','googlenet']  
        },
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'data_aug': {
            'values': ['Yes','No']
        }
    }
}


In [7]:
import wandb

wandb.login(key = 'b5d1fbca9d5170f54415e9c5a70ef09cee7a0aec')



# Initialize Sweep
sweep_id = wandb.sweep(sweep_config, project="MA23M021_A2_Part-B")

# Run the sweep agent
wandb.agent(sweep_id, function=train_with_wandb, count=50)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: v3u06a5x
Sweep URL: https://wandb.ai/ma23m021-iit-madras/MA23M021_A2_Part-B/sweeps/v3u06a5x


wandb: Agent Starting Run: 2q1epvwe with config:
wandb: 	data_aug: Yes
wandb: 	epochs: 5
wandb: 	freeze_percent: 0.9
wandb: 	lr: 0.001
wandb: 	model_name: resnet
wandb: WARNING Ignoring project 'MA23M021_A2_Part-B' when running a sweep.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1 | Train Loss: 1.4225 | Train Acc: 54.48% | Val Loss: 1.1021 | Val Acc: 63.88%


...

Epoch 7 | Train Loss: 0.6826 | Train Acc: 78.10% | Val Loss: 0.8338 | Val Acc: 72.24%


Epoch 8 | Train Loss: 0.6333 | Train Acc: 79.76% | Val Loss: 0.8306 | Val Acc: 72.39%


Epoch 9 | Train Loss: 0.5871 | Train Acc: 81.47% | Val Loss: 0.8283 | Val Acc: 71.94%


Epoch 10 | Train Loss: 0.5496 | Train Acc: 83.35% | Val Loss: 0.8247 | Val Acc: 71.69%


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▄▅▆▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_accuracy,▁▃▅▆▇▇███▇
val_loss,█▅▃▂▂▁▁▁▁▁
epoch,10
train_accuracy,83.35
train_loss,0.54956
val_accuracy,71.68584
val_loss,0.82473


In [7]:
import wandb

wandb.login(key='b5d1fbca9d5170f54415e9c5a70ef09cee7a0aec')

# with one strategy.....................................
config = {
    'model_name': 'resnet',
    'freeze_percent': 0.8,
    'epochs': 10,
    'lr': 0.0001,
    'data_aug': 'No'
}

wandb.init(project="MA23M021_A2_Part-B", name="With one strategy", config=config)

train_with_wandb()

wandb.finish()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch 1 | Train Loss: 1.9126 | Train Acc: 45.88% | Val Loss: 1.5861 | Val Acc: 65.08%


Epoch 2 | Train Loss: 1.4080 | Train Acc: 66.76% | Val Loss: 1.2636 | Val Acc: 67.48%


Epoch 3 | Train Loss: 1.1666 | Train Acc: 70.24% | Val Loss: 1.1017 | Val Acc: 69.78%


Epoch 4 | Train Loss: 1.0374 | Train Acc: 71.89% | Val Loss: 1.0068 | Val Acc: 71.24%


...

Epoch 7 | Train Loss: 0.8778 | Train Acc: 74.29% | Val Loss: 0.8828 | Val Acc: 73.69%


Epoch 8 | Train Loss: 0.8452 | Train Acc: 74.49% | Val Loss: 0.8584 | Val Acc: 73.34%


Epoch 9 | Train Loss: 0.8224 | Train Acc: 74.74% | Val Loss: 0.8495 | Val Acc: 72.64%


Epoch 10 | Train Loss: 0.8035 | Train Acc: 75.40% | Val Loss: 0.8334 | Val Acc: 73.79%


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▆▇▇▇▇████
train_loss,█▅▃▂▂▂▁▁▁▁
val_accuracy,▁▃▅▆▇▇██▇█
val_loss,█▅▃▃▂▂▁▁▁▁
epoch,10
train_accuracy,75.4
train_loss,0.80351
val_accuracy,73.78689
val_loss,0.8334
